# 4. Building Good Training Sets - Data Preprocessing

 데이터의 질과 데이터가 가지는 유용한 정보의 양은 기계학습 알고리즘을 잘 학습시키는데 있어 매우 중요하다. 따라서 데이터를 알고리즘에 넣기 전에 충분히 전처리를 해 주는것이 필수적이다. 이 챕터에서는 그러한 전처리 테크닉에 대해서 알아볼 것이다. 

우리가 배울 주제들은 다음과 같다.
- Missing value를 제거하고 새로 넣어 주는 것
- Categorical data를 머신러닝 알고리즘에 맞게 형태를 변환하는 것
- 모델 구축을 위해 관련된 변인들을 선택하는 것


## Dealing with missing data

현실에서 우리가 가지는 샘플은 여러가지 이유 때문에 한개 혹은 더 많은 value들이 비어있는 경우가 많다. 이러한 missing value들은 주로 비어있는 space로 나타나게 되거나, 'NaN'(Not a Number)로 제시된다.

불행히도, 만일 우리가 missing value를 무시한다면 대부분의 computational tool들은 missing value를 처리하지 못하거나, 예측 불가능한 결과를 내놓게 된다. 따라서 이러한 missing value를 처리하는것이 중요하다. 그 전에 csv 파일을 이용한 예시로 문제를 이해해 보자.

In [89]:
import pandas as pd
from io import StringIO
csv_data = '''A, B, C, D
            1.0, 2.0, 3.0, 4.0
            5.0, 6.0,,8.0
            10.0, 11.0, 12.0'''
df = pd.read_csv(StringIO(csv_data))
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


앞의 코드를 이용해, 우리는 csv포맷의 데이터를 `pandas`의 `Dataframe`으로 읽어왔다. 표를 보게되면, NaN으로 표시된 missing value가 있는 것을 알 수 있다. `StringIO` 함수는 단순히 설명을 위해 사용되었다. 이 함수는 `csv_data`에 string을 `pandas`의 `DataFrame`으로 읽어오게 할 수 있게 해주기 때문이다.

더 큰 `DataFrame`에서는 일일히 missing value를 찾는것이 굉장히 힘든 작업이 될 수 있다. 이런 케이스에는 `isnull`메소드를 이용하여 Boolean 값으로 어떠한 열이 missing value가 없는지(0, False), 혹은 있는지(1, True) `DataFrame`으로 나타낼 수 있다. `Sum`메소드를 이용해, 우리는 다음과 같이 각 열에 missing value가 몇개나 있는지 찾아낼 수 있다.

In [34]:
df.isnull().sum()

A     0
 B    0
 C    1
 D    1
dtype: int64

이런 방법을 이용해 각 열당 얼마만큼의 missing value를 가지는지 셀 수 있다. 다음 소주제는 우리가 이러한 missing data를 어떻게 처리할 것인지에 대한 다양한 전략들에 대한 것이다.

## Eliminating samples or features with missing values

Missing data를 처리하는 가장 쉬운 방법은 단순히 해당하는 열이나 행을 데이터셋에서 지워 버리는 것이다. `dropna`메소드를 통해 우리는 이를 시행할 수 있다.

In [35]:
df.dropna()

,A,B,C,D
0,1.0,2.0,3.0,4.0


비슷하게, `axis` argument를 1로 설정하여 1개 이상의 NaN을 가진 행을 제거할 수 있다.

In [22]:
df.dropna(axis=1)

,A,B
0,1.0,2.0
1,5.0,6.0
2,0.0,11.0


`dropna`메소드는 추가적으로 여러가지의 편리한 parameter들을 제공한다. 

In [ ]:
# 모든 열이 NaN인 행들만 제거
df.dropna(how='all')

# 4개 이상의 non-NAN 값을 가지고 있지 않은 행을 제거
df.dropna(thresh=4)

# 특정 열에 나타나는 NaN값에만 한정해 행 제거(여기서는 'C')
df.dropna(subset=['C'])

Missing data를 제거하는 것이 간편해 보이지만, 이 방법은 명백한 단점들을 가지고 있다. 예를 들어서, 너무 많은 샘플을 제거해 정확한 분석이 되지 않을 수 있다. 또한, 너무 많은 feature columns를 제거할 경우 우리의 분류 알고리즘이 class들을 제대로 분석하는데 필요한 정보들을 잃을 수 있는 리스크도 있다. 다음 주제에서, 우리는 missing value를 처리하는데 있어 가장 많이 사용되는 기법중 하나인 **interpolation**이라는 기법을 살펴 볼 것이다.

## Imputing missing values

종종 단순히 샘플을 제거하거나 해당 열을 통째로 없애는 것은 불가능한데, 우리가 가치있는 정보를 너무 많이 잃을 수 있기 때문이다. 이러한 경우에는 interpolation 기법으로 우리 데이터 안의 다른 훈련 데이터셋을 이용해 missing value를 추정할 수 있다. 가장 흔한 interpolation 기법은 **mean imputation**으로, missing value들을 해당 열의 평군값으로 대체하는 것이다. Scikit-learn의 `Imputer`클래스를 이용해 우리는 이를 쉽게 할 수 있다.

In [91]:
from sklearn.preprocessing import Imputer
imr = Imputer(missing_values ='NaN', strategy='mean', axis=0)
imr = imr.fit(df)
imputed_data = imr.transform(df.values)
imputed_data

array([[  1. ,   2. ,   3. ,   4. ],
       [  5. ,   6. ,   7.5,   8. ],
       [ 10. ,  11. ,  12. ,   6. ]])

여기서 우리는 각각의 NaN값들은 해당되는 열의 평균값으로 대체하였다. 우리가 세팅을 `axis=0`에서 `axis=1`로 바꾼다면, 열이 아닌 행의 평균값을 넣어주게 된다. `strategy` parameter에 대한 다른 값으로는 `median`또는 `most_frequent`가 있다. `most_frequent`같은 경우 category value을 넣어주는데에 유용하다.

## Understanding the scikit-learn estimator API

이전 섹션에서, scikit-learn의 `Imputer`라는 클래스를 이용해 우리는 데이터셋의 missing values를 찾아주었다. `Imputer` 클래스는 scikit-learn의 **transformer**클래스(데이터 변환에 사용)에 속해있다. ~~가장 중요한 두개의 메소드는 `fit`과 `transform`으로, `fit`메소드는 훈련 데이터로부터 parameter들을 학습시키는데에, `transform`메소드는 그러한 parameter들로 데이터를 변환하는데 사용된다. 변환시킬 데이터들은 모델을 학습시키기 위해 사용된 데이터들의 변수들과 같은 변수를 가져야 한다. ~~


<img src="ch4_img1.png" alt="Drawing" style="width: 600px;"/>

챕터3에서 우리가 사용했던 classifiers는 변환 클래스와 개념적으로 매우 비슷한 scikit-learn estimator에 속해있다. 나중에 확인하겠지만, Estimators는 `predict`메소드를 가질수도 있고 `transform`메소드를 가질 수도 있다. 기억하듯, 우리는 분류를 위한 estimator들을 훈련시킬때 `fit`메소드를 사용해 모델의 parameter들을 학습시켰다. 하지만, 지도 학습에서 우리는 모델을 fitting시키기 위해 클래스 라벨을 추가적으로 제공했다. 이러한 클래스 라벨은 새로운 데이터 샘플을 `predict`메소드를 통해 예측할때 사용될 수 있었다.

<img src="ch4_img2.png" alt="Drawing" style="width: 600px;"/>

## Handling categorical data

이제까지 우리의 전처리 대상은 nummerical value들이었다. 하지만 실제 세상의 데이터는 보통 한가지 이상의 categorical feature를 가지고 있다. 우리는 categorical data를 대할때 **nominal** 그리고 **ordinal** features 에 대해 구분할 필요가 있다. Ordinal features는 순서를(ordering) 가지거나 분류할(sorting) 수 있는 categorical value들로 이해할 수 있다. 티셔츠 사이즈가 그렇다(XL>L>M). 대조적으로, nominal features들은 어떠한 순서를 가지지 않는다. 예를 들어서, 빨간색 티셔츠는 파란색 티셔츠 보다 크지 않다.

문제를 설명하기 위해, 새로운 데이터 프레임을 만들어 보자.

In [105]:
import pandas as pd
df = pd.DataFrame([
    ['green', 'M', 10.1, 'class1'],
    ['red', 'L', 13.5, 'class2'],
    ['blue', 'XL', 15.3, 'class1']])
df.columns = ['color', 'size', 'price', 'classlabel']
df

,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


*여기서 분류를 위해 사용되는 학습 알고리즘은 ordinal information클래스 라벨을 사용하지 않는다.

## Mapping ordinal features

학습 알고리즘이 ordinal features을 잘 해석하는지 확인하기 위해, 우리는 범주형 문자열을 integer로 바꿀 필요가 있다. 불행히도, 크기 순서에 맞게 이를 수행하는 편리한 함수는 없다. 밑의 예시에서는 XL = L+1 = M+2를 가정한다.

In [106]:
size_mapping = {
    'XL' : 3,
    'L' : 2,
    'M' : 1
}
df['size'] = df['size'].map(size_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


inter 값들을 다시 원래의 문자열로 돌리기 위해서는, `inv_size_mapping = {v:k for k, v in size_mapping.items()}`과 같은 reverse-mapping 딕셔너리를 사용하면 된다. 이 방법은 우리가 사용했던 `size_mapping`딕셔너리와 비슷한 pandas의 `map`메소드로도 사용될 수 있다.

## Encoding class labels

많은 기계학습 라이브러리들은 클래스 라벨이 integer값으로 인코딩 되어 있어야 사용할 수 있다. scikit-learn의 분류를 위한 대부분의 estimator들은 내부적으로 클래스 라벨을 integer로 변환하지만, 기술적 결함을 피하기 위해 클래스 라벨은 되도록 integer로 바꾸어 놓는게 좋다. 클래스 라벨을 인코드 하기 위해, 이전에 했던 방법과 비슷한 방법을 취할 수 있다. 다시, 클래스 라벨은 ordinal이 *아니며*, 어떤 숫자들이 각각의 클래스 라벨에 지정되었는지는 상관이 없다. 

In [107]:
import numpy as np
class_mapping = {label:idx for idx, label in
                 enumerate(np.unique(df['classlabel']))}
class_mapping


{'class1': 0, 'class2': 1}

다음으로 mapping dictionary를 이용해 클래스 라벨을 integer로 바꿔준다.

In [108]:
df['classlabel'] = df['classlabel'].map(class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,0
1,red,2,13.5,1
2,blue,3,15.3,0


매핑 딕셔너리의 key-value쌍을 두고 원래의 문자열로 바꿔주기 위해 사용할수 있다.

In [109]:
inv_class_mapping = {v: k for k, v in class_mapping.items()}
df['classlabel'] = df['classlabel'].map(inv_class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


다른 방법으로, scikit-learn의 `LabelEncoder`클래스를 사용하면 같은 결과를 얻을 수 있다.

In [143]:
from sklearn.preprocessing import LabelEncoder
class_le = LabelEncoder()
y = class_le.fit_transform(df['classlabel'].values)
y

array([0, 1, 0])

`fit_tramsform`메소드는 `fit`과 `transform`을 따로 불러오는 간단한 방법이다. 또한, `inverse_transform`메소드를 사용해 integer 클래스 라벨을 원래의 문자열로 바꿔주는데 사용할 수 있다.

In [111]:
class_le.inverse_transform(y)

array(['class1', 'class2', 'class1'], dtype=object)

## Performing one-hot encoding on nominal features

이전 섹션에서, 우리는 ordinal size feature를 integer로 변환하기 위해 간단한 dictionary-mapping 방법을 사용했다. scikit-learn의 estimator들이 클래스 라벨을 특정한 순서 없이 인식하기 때문에, `LabelEncoder`라는 편리한 클래스를 사용할 수도 있었다. 우리의 `color`열 또한 비슷한 방법으로 변환해 줄 수 있다.

In [141]:
X = df[['color', 'size', 'price']].values
color_le = LabelEncoder()
X[:, 0] = color_le.fit_transform(X[:, 0])
X

array([[1, 1, 10.1],
       [2, 2, 13.5],
       [0, 3, 15.3]], dtype=object)

앞의 코드를 실행함으로써 X 어레이는 새로운 `color`값들을 갖게 되었다.

- blue -> 0
- green -> 1
- red -> 2

만약 이 상태에서 바로 우리의 classifier에 데이터를 넣게 되면, 우리는 범주형 데이터를 다루는데 있어 가장 흔한 실수중 하나를 하게 된다. color값들이 특정한 순서를 갖지 않더라도, 학습 데이터는 *녹색* 을 *파란색* 보다 큰 값으로, *빨간색* 은 *녹색*보다 큰 값으로 인식할 것이다. 이러한 인식이 정확하지 않더라도, 알고리즘은 꽤나 쓸만한 결과를 내놓을 수 있다. 하지만 그러한 결과는 최선은 아니다.


이 문제를 다루기 위해 **one-hot encoding**이라고 불리는 기법을 이용한다. 이 방법의 아이디어는 nomial feature column에 있는 고유한 값들로 새로운 **dummy feature**를 만드는 것이다. 우리는 `color` feature로 새로운 세개의 features를 만들 수 있다(파란색, 녹색, 빨간색). 이 features에는 Binary value가 사용된다. 예를 들어서, 파란색은 `blue=1, green=0, red=0`으로 나타내어 질 수 있다. 이를 위해 `scikit-learn.preprocessing`모듈의 `OneHotEncoder`를 사용한다.

In [145]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(categorical_features=[0])
ohe.fit_transform(X).toarray()

array([[  0. ,   1. ,   0. ,   1. ,  10.1],
       [  0. ,   0. ,   1. ,   2. ,  13.5],
       [  1. ,   0. ,   0. ,   3. ,  15.3]])

`OneHotEncoder`를 initialize할때, 우리는 `categorical_features` parameter를 통해 바꾸기를 원하는 열을 정해준다(여기서 color는 0열). 원래, `OneHotEncoder`는 `transform`메소드를 사용할때 sparse matrix를 리턴한다. 그리고 그 sparse matrix를 시각화를 위해 regular(dense) NumPy 어레이로 바꿔주기 위해 `toarray` 메소드를 사용하였다. `toarray`메소드를 사용하지 않고 바로 NumPy 어레이로 리턴하기 위해서는 `OneHotEncoder(..., sparse = False)`로 initialize 하면 된다.

더 편리한 방법은 pandas의 `get_dummies`메소드를 이용한 방법이다. `DataFrame`에 적용되는 이 메소드는 나머지 열들은 놔두고 문자열 열들만을 바꾼다.

In [125]:
pd.get_dummies(df[['price', 'color', 'size']])

,price,size,color_blue,color_green,color_red
0,10.1,1,0,1,0
1,13.5,2,0,0,1
2,15.3,3,1,0,0


## Partitioning a dataset in training and test sets

챕터1, 챕터3에서 훈련과 테스팅 데이터셋을 나누는 것에 대해 간략하게 소개하였다. 우리는 테스트셋이 우리가 모델을 현실에 적용하기 전에 하는 *마지막 테스트* 로 이해할 수 있음을 배웠다. 우리는 이 섹션에서 새로운**Wine** 데이터셋을 사용할 것이다. 데이터를 전처리 한 후, 데이터셋의 차원을 감소시키기 위한 feature selection에 대해 알아 볼 것이다.

Wine dataset source: https://archive.ics.uci.edu/ml/datasets/Wine

pandas 라이브러리를 이용해 이 데이터를 바로 읽어온다.

In [128]:
df_wine = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data',
                      header=None)

df_wine.columns = ['Class label', 'Alcohol',

                    'Malic acid', 'Ash',

                    'Alcalinity of ash', 'Magnesium',

                    'Total phenols', 'Flavanoids',

                    'Nonflavanoid phenols',

                    'Proanthocyanins',

                    'Color intensity', 'Hue',

                    'OD280/OD315 of diluted wines',

                    'Proline']

print('Class labels', np.unique(df_wine['Class label']))

df_wine.head()

Class labels [1 2 3]


,Class label,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


샘플들은 1,2,3중 하나의 클래스 라벨을 가지며, 이 라벨은 다른 타입의 포도를 의미한다.

이 데이터셋을 test와 training 데이터로 랜덤하게 나누는 빠른 방법은 scikit-learn의 cross_validation submodule의 `train_test_split`함수를 이용하는 것이다.

In [137]:
from sklearn.cross_validation import train_test_split
X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

## Brining features onto the same scale

**Feature scaling**은 전처리 파이프라인에 있어 매우 중요한 과정이다. Decision trees와 random forest를 제외한 거의 대부분의 기계학습 알고리즘은 이 과정을 필요로 한다. 특히나 챕터2에서 보았듯 **gradient descent**를 적용할때 그렇다.

Feature scaling의 중요성은 간단한 예로서 증명할 수 있다. 예를 들어서, 첫번째 feature는 1에서 10까지의 값을 가지고 두번째는 1에서 100,000까지의 값을 가진다고 하자. 우리가 챕터2 에서 볼 수 있는 **Adaline**의 squared error function을 생각해보면, 두번째 feature의 가중치를 최적화하는데에 많은 시간이 소요될 것이다. 이는 두번째 feature가 큰 에러를 가지기 때문이다. **KNN**또한, 두번째 feature때문에 적절한 거리를 산출할 수 없을 것이다.

**Normalization**과 **Standardization**은 이를 해결하기 위해 많이 쓰이는 방법이다. normalization은 보통 feature들을 [0,1]사이의 숫자로 rescale 하는것을 말한다(min-max scaling). 식은 다음과 같다.

<img src="ch4_img3.png" alt="Drawing" style="width: 250px;"/>

scikit-learn을 이용해 min-max scaling이 가능하다.

In [139]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
X_train_norm = mms.fit_transform(X_train)
X_test_norm = mms.transform(X_test)

min-max scaling을 이용한 normalization은 보통 bounded interval안에 있는 값들이 필요할때 유용하게 쓰이며, standardization은 더 많은 기계학습 알고리즘에 더 쓸모있다. 로지스틱 회귀나 SVM와 같은 선형모델은 가중치를 0이나 0에 가까운 값으로 초기화하기 때문이다. standardization을 이용해, 우리는 feature column을 평균이 0인 동시에 표준편차가 1인 정규분포로 변환할 수 있으며, 이는 가중치를 학습하기에 유리하다. 또한 standardization은 outlier에 대한 유용한 정보를 유지하면서도 알고리즘이 outlier들에 대해 덜 민감하게 만들며, 이는 min-max scaling과 다른 특징점이다.

식은 다음과 같다.

<img src="ch4_img4.png" alt="Drawing" style="width: 250px;"/>

다음 테이블은 두 방법에 따라 산출한 값들이다. 

<img src="ch4_img5.png" alt="Drawing" style="width: 500px;"/>

`MinMaxScaler`와 비슷하게, scikit-learn은 standardization에 대한 클래스 또한 제공한다.

In [140]:
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.transform(X_test)

nomarlization과 같이, `StandardScaler`또한 훈련 데이터에 *한번만* 적용시킨다.